In [1]:
%matplotlib ipympl

import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from scipy.integrate import cumtrapz
from scipy.signal import welch

In [2]:
# Load data
xls_file = 'data/Rover_II_Current_Mag_Hourly_Avg_pad.xlsx'
df = pd.read_excel(xls_file,parse_dates=[6])

# Datetime index
df = df.set_index('Date_time_R')

# Create u and v components with units m/s
df['u'] = df['Easting (cm/sec)']/100
df['v'] = df['Northing (cm/sec)']/100

ds_alt = xr.open_dataset('data/jpl_sla_uv_timeseries.nc')

nc_file_tg = '../../Projects/CCS_sea_level/data/noaa_tide_gauge.nc'
ds_tg = xr.open_dataset(nc_file_tg)

In [3]:
ds_tg

<xarray.Dataset>
Dimensions:         (position: 8, time: 372528)
Coordinates:
  * time            (time) datetime64[ns] 1975-01-01 ... 2017-06-30T23:00:00
    lat             (position) float64 ...
    lon             (position) float64 ...
Dimensions without coordinates: position
Data variables:
    station_name    (position) object ...
    waterlevel      (time, position) float64 ...
    station_id      (position) object ...
    waterlevelfilt  (time, position) float64 ...

In [4]:
df.head()

,Date_time_Merge_hour,Current_mag,Easting (cm/sec),Northing (cm/sec),Minutes,Seconds,ABSOLUTE Direction,Heading,u,v
Date_time_R,,,,,,,,,,
2011-11-20 13:11:20,2011-11-20 13:00:00,7.712734,-3.678000,-1.354000,10.8,32.0,69.789580,69.789580,-0.036780,-0.013540
2011-11-20 13:56:13,2011-11-20 14:00:00,1.711170,1.253333,-0.853333,16.0,13.0,55.750967,304.249033,0.012533,-0.008533
2011-11-20 14:55:30,2011-11-20 15:00:00,2.024802,1.080000,-1.430000,25.0,30.0,37.061735,322.938265,0.010800,-0.014300
2011-11-20 15:59:28,2011-11-20 16:00:00,3.567306,-1.413333,-3.036667,19.0,28.0,24.958334,24.958334,-0.014133,-0.030367
NaT,2011-11-20 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
ds_alt

<xarray.Dataset>
Dimensions:    (time: 187)
Coordinates:
    Longitude  float32 ...
    Latitude   float32 ...
    lon        float32 ...
    lat        float32 ...
  * time       (time) datetime64[ns] 2015-11-26T12:00:00 ... 2018-06-13T12:00:00
Data variables:
    va         (time) float64 ...
    sla_err    (time) float64 ...
    ua         (time) float64 ...
    sla        (time) float64 ...

In [33]:
plt.figure()
plt.plot(df.index,df['u']);
#plt.plot(ds_alt['time'],ds_alt['sla'])
plt.ylabel('m/s')
plt.title('eastward velocity')
plt.plot(ds_tg['time'],ds_tg['waterlevelfilt'][:,5]/3)
plt.xlim([np.datetime64('2015-01-01'),np.datetime64('2015-03-01')])

plt.figure()
plt.plot(df.index,df['v']);
#plt.plot(ds_alt['time'],ds_alt['va']/10)
plt.ylabel('m/s')
plt.title('northward velocity')
plt.plot(ds_tg['time'],ds_tg['waterlevelfilt'][:,5]/3)
plt.xlim([np.datetime64('2016-01-01'),np.datetime64('2016-03-01')])


/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

(735964.0, 736024.0)

In [7]:
# t1=np.datetime64('2018-02-01')
# t2=np.datetime64('2018-02-15')

# plt.figure()
# plt.plot(ds['time'],ds['Eas'][:,15]);
# plt.plot(ds['time'],ds['Nor'][:,15]);
# plt.ylabel('m/s')
# plt.xlim([t1,t2])
# plt.ylim([-0.15,0.15])
# plt.legend(['East','North'])
# plt.title('full velocity, '+str(t1)+'-'+str(t2)+', bin 15')

In [8]:
# t1=np.datetime64('2018-02-01')
# t2=np.datetime64('2018-02-15')

# plt.figure()
# plt.plot(ds['time'],ds['Eas'][:,15]-ds['Eas_tide'][:,15]);
# plt.plot(ds['time'],ds['Nor'][:,15]-ds['Nor_tide'][:,15]);
# plt.legend(['Bin 15','Bin 1'])
# plt.ylabel('m/s')
# plt.ylim([-0.15,0.15])
# plt.xlim([t1,t2])
# plt.legend(['East','North'])
# plt.title('detided velocity, '+str(t1)+'-'+str(t2)+', bin 15')

In [9]:
# Eas = np.array(ds['Eas'])
# Nor = np.array(ds['Nor'])
# Eas_tide = np.array(ds['Eas_tide'])
# Nor_tide = np.array(ds['Nor_tide'])

# N = len(Eas)
# fE,SE = welch(Eas[:,5],fs=288.,nperseg=N/6,window='hanning')
# fN,SN = welch(Nor[:,5],fs=288.,nperseg=N/6,window='hanning')

# #rotary spectrum
# fr,Sr = welch(Eas[:,5]+1j*Nor[:,5],
#               fs=288.,nperseg=N/6,
#               window='hann',return_onesided=False)
# icw, = np.where(fr<0)
# iccw, = np.where(fr>0)
# fcw = -fr[icw]
# Scw = Sr[icw]
# fccw = fr[iccw]
# Sccw = Sr[iccw]

In [10]:
# plt.figure()
# plt.loglog(fcw,Scw)
# plt.loglog(fccw,Sccw)
# plt.legend(['CW','CCW'])

### Future processing steps
* Fill small gaps
* QC based on correlation, percent good, etc.

### Ideas for future analysis

* Wavelet analysis: isolate time variability of inertial band
* Relate inertial currents to local wind
* Complex correlation of low-pass filtered currents
* Is low frequency variability due to eddies? Relate to satellite altimeter data
* Explore relationship to local and remote wind stress curl
* Read Clarke paper on ("too fast") Rossby wave propagation in CalCOFI region
* Compare tidal currents to OSU model
* Progressive vector diagram
* Rover data - Lomb-Scargle periodogram - analysis of eddy variability at O(100 day) periods - see Stabeno et al.